#Capstone Project

###*Lawrence Sheed*

---


In [1]:
#Disable pip's bloody annoying warnings
%env _PIP_LOCATIONS_NO_WARN_ON_MISMATCH=1 

#disable python warnings
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

#install libraries needed
!pip install beautifulsoup4
!pip install requests
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install geopy
!pip install geocoder

#import libraries
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import json
import folium 
from bs4 import BeautifulSoup  

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
    

from sklearn.cluster import KMeans
from IPython.display import display_html
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import geocoder # import geocoder

print('Libraries imported.')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



env: _PIP_LOCATIONS_NO_WARN_ON_MISMATCH=1
Libraries imported.


In [2]:
# URL https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

#Read table with soup
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(url).text
soup= BeautifulSoup (req,'lxml')
table = soup.find ('table')

#disable debug code to see what was up with the tables
#print (table)
#myTable=str(soup.table)
#display_html(myTable,raw=True)



In [3]:

table_contents=[]
#table=soup.find('table')

for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
        

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A','East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business','EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto','MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

print (df.head())
print()

print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)


  PostalCode           Borough                      Neighborhood
0        M3A        North York                         Parkwoods
1        M4A        North York                  Victoria Village
2        M5A  Downtown Toronto         Regent Park, Harbourfront
3        M6A        North York  Lawrence Manor, Lawrence Heights
4        M7A      Queen's Park     Ontario Provincial Government

The dataframe has 15 boroughs and 103 neighborhoods.


##Import the CSV and Merge

In [4]:
#import the CSV
geo_coords = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')
geo_coords.head()

#merge the two 
geo_coords.rename(columns={'Postal Code':'PostalCode'},inplace=True)
merged = pd.merge(df,geo_coords,on='PostalCode')
merged.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


##Check out Toronto's Stats

In [10]:
toronto = merged[merged['Borough'].str.contains('Toronto',regex=False)]
toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
35,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106


##Map time!

In [11]:
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(toronto['Latitude'],toronto['Longitude'],toronto['Borough'],toronto['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='red',
    fill=True,
    fill_color='seagreen',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

##Create Cluster using KMEANS per neighbourhood in Toronto

In [13]:
k=5
toronto = merged[merged['Borough'].str.contains('Toronto',regex=False)]
toronto_clustering = toronto.drop(['PostalCode','Borough','Neighborhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)


print ("KMEANS array", kmeans.labels_)

toronto.insert(0, 'KMeansCluster', kmeans.labels_)

toronto

KMEANS array [4 4 4 1 4 4 0 4 3 1 4 0 1 4 0 1 4 1 2 2 2 2 3 2 0 3 2 0 3 2 0 2 0 4 4 4 4
 4 1]


,KMeansCluster,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
9,4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,1,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,0,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,4,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
31,3,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
35,1,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106


##Using KMeans clustering to draw MAP

In [8]:
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(toronto['Latitude'], toronto['Longitude'], toronto['Neighborhood'], toronto['KMeansCluster']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters
